CIFAR-10 is a subset of an 80 million image collection, with 60,000 images in total. The training data consists of 50k images, while the test data contains 10k images. The CIFAR-10 dataset consists of 10 categories with each category containing 6000 images.


Our focus in this notebook will be on PyTorch for building and training on the CIFAR10 Dataset. PyTorch simplifies the learning process.


# Importing the libraires required to run the CIFAR10 pytorch program

In [56]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.transforms as transforms
import torchvision.models as models
import time
import torchvision
import torch.optim as optim

# Extract and download the dataset

the dataset has 100 batches and each batch will have 60000/100 = 6000 images and the test batch will have total of 10 k images

In [65]:
def reshapedataset():
    transform_train = transforms.Compose([transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #Normalize all the images
                               ])
    transform = transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])
    #transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
   # batch_size = 128
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                                  shuffle=True, num_workers=2 , pin_memory=True)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                               download=True, transform=transform_train)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                                 shuffle=False, num_workers=2, pin_memory=True)

    return trainset, testset



# Set up the device and set the parameters


In [58]:
#set the device 

torch.cuda.is_available()
print('torch.cuda.is_available():', torch.cuda.is_available())
torch.cuda.current_device()
torch.cuda.get_device_name()
device=torch.device("cuda:0" if torch.cuda.is_available else "cpu")
device

torch.cuda.is_available(): True


device(type='cuda', index=0)

In [59]:
## set parameters
inputsize = 3072
numofclasses =10
learning_rate = 0.001
batch_size = 100
num_epochs = 20



# Building Neural Network Model

We have built a Neural network Model of 5 hidden layers. Since we have colour images our dataset will be divided into numeric values of RGB.The first layer is with input node [3072,1024,512,64,64,10]. Forward pass will use Rectified Activation unit(ReLu) which is easy to compute. To avoid activating all the nodes a transistion node is used.


In [60]:
#Create fully connected Network

class NN(nn.Module): #inherit from nn.module#
    
    def __init__(self,inputsize,Layer1,Layer2,numofclasses,activation_functions):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(inputsize,1024)# input layer and first layer
       
        self.fc2 =  nn.Linear(1024,512)#second layer and output layer#
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(512,64)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(64,64)
        self.relu4 = nn.ReLU()

        self.fc5 = nn.Linear(64,10)
        self.activation_functions = activation_functions
    def forward(self,xb):
        out = xb.view(xb.size(0), -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)
        out = F.relu(out)
        out = self.fc5(out)
        return out
    


# Predicting the accuracy

Accuracy is predicted by number of correct prediction out of total predictions

In [61]:

def checkaccuracy(loader, model,nameofdataset):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x , y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            #x= x.reshape(x.shape[0],-1)
            
            scores = model(x)
            _, predictions = scores.max(1) #max of the second dimension,gives the index
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f'Got{num_correct/num_samples} with accuracy {float(num_correct)/float(num_samples)*100}', nameofdataset)
        
    model.train()
    

# Training the Model

Dataloader takes full advantage of our custom dataset class that inherits Dataset from torch library
 
 We have used Adam optimizer with betas and weight decay value.
 
 we first check the data for training set and validate the accuracy with the testing set

In [67]:


# Initialise Artifical Neural Network

model = NN(inputsize = inputsize,Layer1 = 1024, Layer2 = 512 , numofclasses = numofclasses, activation_functions = activation_functions).to(device)
#Loss and optimiser
loss_criteria = nn.CrossEntropyLoss()
betas=(0.9,0.995)
weight_decay=5e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=betas,weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum =0.9)
trainloader , testloader = reshapedataset()
train_loader = DataLoader(trainloader , batch_size= batch_size, shuffle = False )
test_loader = DataLoader(testloader , batch_size= batch_size, shuffle = False )
for epoch in range(num_epochs):
    #check for each each batch loader from a tuple
    train_losses = []
    train_accuracy = []
    for batch, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(data.shape)
        #reshape to single vector , correct shape
        #trainloader= DataLoader(train_subset, batch_size=batch_size, shuffle=True) 

        data = data.reshape(data.shape[0],-1)
        #print(data.shape)
        scores = model(data)
        
        loss = loss_criteria(scores,targets)
        #print(scores)
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        
        #adam
        optimizer.step()
checkaccuracy(train_loader,model,"Train")
checkaccuracy(test_loader,model,"Test")
       


Files already downloaded and verified
Files already downloaded and verified
Got0.6381799578666687 with accuracy 63.818 Train
Got0.5169000029563904 with accuracy 51.690000000000005 Test


# Conclusions - 1

As we can see above the accuracy for CIFAR 10 dataset with Neural network for training dataset is 63 % and testing is 51%. there is a difference of 12%. We can infer that Neural network is not successful in its prediction of images

Future to this we have tried with less Layers but still there is no much improvement in the accuracy.

Firstly let's try to tune the dataset and run the model to check how it is effecting the accuracy

In [70]:
 def reshapedataset():
        transform_train = transforms.Compose([transforms.ToTensor(), # comvert the image to tensor so that it can work with torch
                                      transforms.Normalize((0.4913997551666284, 0.48215855929893703, 0.4465309133731618),
                                  (0.24703225141799082, 0.24348516474564, 0.26158783926049628))
                                ])
        transform = transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.4913997551666284, 0.48215855929893703, 0.4465309133731618),
                                      (0.24703225141799082, 0.24348516474564, 0.26158783926049628))])


        trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                    download=True, transform=transform_train)
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                                      shuffle=True, num_workers=2 , pin_memory=True)

        testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                                   download=True, transform=transform_train)
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                                     shuffle=False, num_workers=2, pin_memory=True)

        return trainset, testset



In [71]:


# Initialise Artifical Neural Network

model = NN(inputsize = inputsize,Layer1 = 1024, Layer2 = 512 , numofclasses = numofclasses, activation_functions = activation_functions).to(device)
#Loss and optimiser
loss_criteria = nn.CrossEntropyLoss()
betas=(0.9,0.995)
weight_decay=5e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate, betas=betas,weight_decay=weight_decay)
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum =0.9)
trainloader , testloader = reshapedataset()
train_loader = DataLoader(trainloader , batch_size= batch_size, shuffle = False )
test_loader = DataLoader(testloader , batch_size= batch_size, shuffle = False )
for epoch in range(num_epochs):
    #check for each each batch loader from a tuple
    train_losses = []
    train_accuracy = []
    for batch, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(data.shape)
        #reshape to single vector , correct shape
        #trainloader= DataLoader(train_subset, batch_size=batch_size, shuffle=True) 

        data = data.reshape(data.shape[0],-1)
        #print(data.shape)
        scores = model(data)
        
        loss = loss_criteria(scores,targets)
        #print(scores)
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        
        #adam
        optimizer.step()
checkaccuracy(train_loader,model,"Train")
checkaccuracy(test_loader,model,"Test")
       


Files already downloaded and verified
Files already downloaded and verified
Got0.6262999773025513 with accuracy 62.629999999999995 Train
Got0.5084999799728394 with accuracy 50.849999999999994 Test


# Conclusions -2 

As we see still there is sigificant difference between Train and test accuracy and no much improvement lets add  dropout of p=0.2 in the Neural network Model

In [72]:
class NN_dropout(nn.Module): #inherit from nn.module#
    
    def __init__(self,inputsize,Layer1,Layer2,numofclasses,activation_functions,p):
        super(NN_dropout,self).__init__()
        #dropoutpercentage = 0.2

        self.fc1 = nn.Linear(inputsize,1024)# input layer and first layer
        self.dropout1 = nn.Dropout(p=p)
        self.fc2 = nn.Linear(1024,512)#second layer and output layer#
        self.dropout2 = nn.Dropout(p=p)
        self.fc3 = nn.Linear(512,64)
        self.dropout3 = nn.Dropout(p=p)
        
        self.fc4 = nn.Linear(64,64)
        self.dropout4 = nn.Dropout(p=p)

        self.fc5 = nn.Linear(64,10)
        self.dropout5 = nn.Dropout(p=p)
        
    def forward(self,xb):
        out = xb.view(xb.size(0), -1)
        out = self.fc1(out)
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        out = F.relu(out)
        out = self.fc4(out)
        out = F.relu(out)
        out = self.fc5(out)
        return out
    

In [76]:


# Initialise Artifical Neural Network

modeldropout = NN_dropout(inputsize = inputsize,Layer1 = 1024, Layer2 = 512 , numofclasses = numofclasses, activation_functions = activation_functions,p=0.2).to(device)
#Loss and optimiser
loss_criteria = nn.CrossEntropyLoss()
betas=(0.9,0.995)
weight_decay=5e-4
optimizer = optim.Adam(modeldropout.parameters(), lr=learning_rate, betas=betas,weight_decay=weight_decay)
#optimizer = optim.SGD(modeldropout.parameters(), lr=learning_rate, momentum = 0.9)

trainloader , testloader = reshapedataset()
train_loader = DataLoader(trainloader , batch_size= batch_size, shuffle = False )
test_loader = DataLoader(testloader , batch_size= batch_size, shuffle = False )
for epoch in range(num_epochs):
    #check for each each batch loader from a tuple
    
    for batch, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(data.shape)
        #reshape to single vector , correct shape
        #trainloader= DataLoader(train_subset, batch_size=batch_size, shuffle=True) 

        data = data.reshape(data.shape[0],-1)
        #print(data.shape)
        scores = modeldropout(data)
        
        loss = loss_criteria(scores,targets)
        #print(scores)
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        
        #adam
        optimizer.step()
            
checkaccuracy(train_loader,modeldropout,"Train")
checkaccuracy(test_loader,modeldropout,"Test")


Files already downloaded and verified
Files already downloaded and verified
Got0.6245799660682678 with accuracy 62.458000000000006 Train
Got0.5072000026702881 with accuracy 50.72 Test



## Lets add a dropout of 0.5 and check how it is effecting the accuracy and make a conclusion on it

In [77]:


# Initialise Artifical Neural Network

modeldropout = NN_dropout(inputsize = inputsize,Layer1 = 1024, Layer2 = 512 , numofclasses = numofclasses, activation_functions = activation_functions,p=0.5).to(device)
#Loss and optimiser
loss_criteria = nn.CrossEntropyLoss()
betas=(0.9,0.995)
weight_decay=5e-4
optimizer = optim.Adam(modeldropout.parameters(), lr=learning_rate, betas=betas,weight_decay=weight_decay)
#optimizer = optim.SGD(modeldropout.parameters(), lr=learning_rate, momentum = 0.9)

trainloader , testloader = reshapedataset()
train_loader = DataLoader(trainloader , batch_size= batch_size, shuffle = False )
test_loader = DataLoader(testloader , batch_size= batch_size, shuffle = False )
for epoch in range(num_epochs):
    #check for each each batch loader from a tuple
    
    for batch, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(data.shape)
        #reshape to single vector , correct shape
        #trainloader= DataLoader(train_subset, batch_size=batch_size, shuffle=True) 

        data = data.reshape(data.shape[0],-1)
        #print(data.shape)
        scores = modeldropout(data)
        
        loss = loss_criteria(scores,targets)
        #print(scores)
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        
        #adam
        optimizer.step()
            
checkaccuracy(train_loader,modeldropout,"Train")
checkaccuracy(test_loader,modeldropout,"Test")


Files already downloaded and verified
Files already downloaded and verified
Got0.6253600120544434 with accuracy 62.536 Train
Got0.5185999870300293 with accuracy 51.85999999999999 Test


# Conclusions - 3

As we see there is no much difference between the dropout [0.2] and [0.5] added to the layers . We can conclude that Neural network is not the apt solution to predict CIFAR images.

We need to build a much higher evaluating model like CNN. 

# Building a CNN model

we have built a CNN model of 6 convo 2d layers,  Relu , Maxpool2d. We have also used the functionality such as BatchNormal and Flatten to tune it.

Lets check the accuracy of the model.

In [78]:
class CnnModel(nn.Module):
    def __init__(self):
        super(CnnModel,self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
            nn.BatchNorm2d(64),
            

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
            nn.BatchNorm2d(256),

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

In [79]:
modelCNN = CnnModel().to(device)
#Loss and optimiser

loss_criteria = nn.CrossEntropyLoss()
betas=(0.9,0.995)
weight_decay=5e-4
optimizer = optim.Adam(modelCNN.parameters(), lr=learning_rate,betas=betas,weight_decay=weight_decay)
#optimizer = optim.SGD(modelCNN.parameters(), lr=learning_rate, momentum = 0.9)

trainloader , testloader = reshapedataset()
train_loader = DataLoader(trainloader , batch_size= batch_size, shuffle = False )
test_loader = DataLoader(testloader , batch_size= batch_size, shuffle = False )

for epoch in range(num_epochs):
    #check for each each batch loader from a tuple
    
    for batch, (data, targets ) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(data.shape)
        #reshape to single vector , correct shape
        #trainloader= DataLoader(train_subset, batch_size=batch_size, shuffle=True) 
        #print(data.shape)
        #data = data.reshape(data.shape[0],-1)
        #print(data.shape)
        scores = modelCNN(data)
        
        loss = loss_criteria(scores,targets)
        #print(scores)
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #adam
        optimizer.step()

checkaccuracy(train_loader,modelCNN,"Train")
checkaccuracy(test_loader,modelCNN,"Test")

Files already downloaded and verified
Files already downloaded and verified
Got0.9612799882888794 with accuracy 96.128 Train
Got0.8316999673843384 with accuracy 83.17 Test


# Conclusions -4

As we see there is drastic improvement in the accuracy of the model by using CNN. but still there is 13 % error rate between the training and the testing set. Lets add dropouts and check the error rate between both. 

In [83]:
class CnnModel(nn.Module):
    def __init__(self,dropout):
        super(CnnModel,self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
            nn.BatchNorm2d(64),
            

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
            nn.BatchNorm2d(128),

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
            nn.BatchNorm2d(256),

            nn.Flatten(), 
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

In [86]:
modelCNN = CnnModel(dropout = 0.2).to(device)
#Loss and optimiser
dropout = 0.2
loss_criteria = nn.CrossEntropyLoss()
betas=(0.9,0.995)
weight_decay=5e-4
optimizer = optim.Adam(modelCNN.parameters(), lr=learning_rate,betas=betas,weight_decay=weight_decay)
#optimizer = optim.SGD(modelCNN.parameters(), lr=learning_rate, momentum = 0.9)

trainloader , testloader = reshapedataset()
train_loader = DataLoader(trainloader , batch_size= batch_size, shuffle = False )
test_loader = DataLoader(testloader , batch_size= batch_size, shuffle = False )

for epoch in range(num_epochs):
    #check for each each batch loader from a tuple
    
    for batch, (data, targets ) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(data.shape)
        #reshape to single vector , correct shape
        #trainloader= DataLoader(train_subset, batch_size=batch_size, shuffle=True) 
        #print(data.shape)
        #data = data.reshape(data.shape[0],-1)
        #print(data.shape)
        scores = modelCNN(data)
        
        loss = loss_criteria(scores,targets)
        #print(scores)
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #adam
        optimizer.step()

checkaccuracy(train_loader,modelCNN,"Train")
checkaccuracy(test_loader,modelCNN,"Test")

Files already downloaded and verified
Files already downloaded and verified
Got0.9057599902153015 with accuracy 90.57600000000001 Train
Got0.8037999868392944 with accuracy 80.38 Test


In [88]:
modelCNN = CnnModel(dropout = 0.3).to(device)
#Loss and optimiser
dropout = 0.5
loss_criteria = nn.CrossEntropyLoss()
betas=(0.9,0.995)
weight_decay=5e-4
optimizer = optim.Adam(modelCNN.parameters(), lr=learning_rate,betas=betas,weight_decay=weight_decay)
#optimizer = optim.SGD(modelCNN.parameters(), lr=learning_rate, momentum = 0.9)

trainloader , testloader = reshapedataset()
train_loader = DataLoader(trainloader , batch_size= batch_size, shuffle = False )
test_loader = DataLoader(testloader , batch_size= batch_size, shuffle = False )

for epoch in range(num_epochs):
    #check for each each batch loader from a tuple
    
    for batch, (data, targets ) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)
        #print(data.shape)
        #reshape to single vector , correct shape
        #trainloader= DataLoader(train_subset, batch_size=batch_size, shuffle=True) 
        #print(data.shape)
        #data = data.reshape(data.shape[0],-1)
        #print(data.shape)
        scores = modelCNN(data)
        
        loss = loss_criteria(scores,targets)
        #print(scores)
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #adam
        optimizer.step()

checkaccuracy(train_loader,modelCNN,"Train")
checkaccuracy(test_loader,modelCNN,"Test")

Files already downloaded and verified
Files already downloaded and verified
Got0.8759199976921082 with accuracy 87.592 Train
Got0.796999990940094 with accuracy 79.7 Test


# Conclusions 

As we clearly see we get the best accuracy of the model when the dropout is 0.2 and the error rate between the data is just 9-10%, we can consider this as our best working model


In [ ]:
def plot_accuracies(history):
    Validation_accuracies = [x['Accuracy'] for x in history]
    Training_Accuracies = [x['train_accuracy'] for x in history]
    plt.plot(Training_Accuracies, '-rx')
    plt.plot(Validation_accuracies, '-bx')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(['Training', 'Validation'])
    plt.title('Accuracy vs. No. of epochs');
plot_accuracies(history)